In [45]:
# Chargement des modules necessaire :

import os

import cv2, numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.preprocessing.image import load_img,img_to_array

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import cv2, numpy as np

In [46]:
# On declare les chemins vers les donnees :

# Image = 'Data'
        
train_data_dir = 'cleaned_data/spectrogram_train_test/Train'
validation_data_dir = 'cleaned_data/spectrogram_train_test/Test'

# Dimmension et path :
img_width, img_height = 224, 224

ImageTRAINCOVID = os.listdir(train_data_dir + '/Cov')
ImageTRAINNORMAL = os.listdir(train_data_dir + '/noncovid')

ImageVALIDATIONCOVID = os.listdir(validation_data_dir + '/Cov')
ImageVALIDATIONNORMAL = os.listdir(validation_data_dir + '/noncovid')

#ImageTESTCOVID = os.listdir(test_data_dir + '/Covid-19')
#ImageTESTNORMAL = os.listdir(test_data_dir + '/spectro_healthy')

print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINNORMAL)) + ' images d\'entrainement de patient non-covid.\n') 

print('Il y a ' + str(len(ImageVALIDATIONCOVID)) + ' images de validation de patient covid.') 
print('Il y a ' + str(len(ImageVALIDATIONNORMAL)) + ' images de validation de patient non-covid.\n')

#print('Il y a ' + str(len(ImageTESTCOVID)) + ' images de test de patient covid.') 
#print('Il y a ' + str(len(ImageTESTNORMAL)) + ' images de test de patient non-covid.')

Il y a 66 images d'entrainement de patient covid.
Il y a 892 images d'entrainement de patient non-covid.

Il y a 29 images de validation de patient covid.
Il y a 315 images de validation de patient non-covid.



In [47]:
# Preprocessing :
# On rescale les images :
datagen = ImageDataGenerator(rescale=1./255)

# On definit la batch size :
batch_size = 32

# On prepare les tableaux de donnees depuis les images :
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

# On prepare les tableaux de donnees depuis les images :
validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=True)

#test_generator_bottleneck = datagen.flow_from_directory(
        #test_data_dir,
        #target_size=(img_width, img_height),
        #batch_size=batch_size,
        #class_mode=None,
        #shuffle=True)




Found 958 images belonging to 2 classes.
Found 344 images belonging to 2 classes.


In [48]:
# Chargement de VGG19 sans la partie fully-connected avec le reseau convolutif entrainer sur imagenet :

model_vgg = applications.VGG19(include_top=False, weights='imagenet')

In [49]:
# On utilse le model VGG16 pour extraire les features de nos images 
# (on fait recupere la sortie du reseau convolutionnel) :
train_features = model_vgg.predict_generator(train_generator_bottleneck, 958 // batch_size)
validation_features = model_vgg.predict_generator(validation_generator_bottleneck, 344 // batch_size)
#test_features = model_vgg.predict_generator(test_generator_bottleneck, 1560 // batch_size)

C:\Users\marti\AppData\Local\Temp\ipykernel_19880\1039380922.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  train_features = model_vgg.predict_generator(train_generator_bottleneck, 958 // batch_size)
C:\Users\marti\AppData\Local\Temp\ipykernel_19880\1039380922.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  validation_features = model_vgg.predict_generator(validation_generator_bottleneck, 344 // batch_size)


In [43]:
# L'opération étant longue on enregistre les features obtenus :

np.save(open('models/trainFeatures_VGG19_AllData.npy', 'wb'), train_features) # ecriture en binaire necessaire
np.save(open('models/validationFeatures_VGG19_AllData.npy', 'wb'), validation_features) # Idem

#np.save(open('models/testFeatures_VGG16_AllData.npy', 'wb'), test_features) # Idem

In [44]:
# Si l'operation à deja été effectuer on charge les features :

train_features = np.load(open('models/trainFeatures_VGG19_AllData.npy', 'rb'))
validation_features = np.load(open('mod'binary'els/validationFeatures_VGG19_AllData.npy', 'rb'))

#test_features = np.load(open('models/testFeatures_VGG16_AllData.npy', 'rb'))

SyntaxError: invalid syntax (2372362487.py, line 4)

In [31]:
train_labels = np.array([0] * 66 + [1] * 892 )
validation_labels = np.array([0] * 29 + [1] * 315 )

In [32]:
# Troisieme modele :
model_top3 = Sequential()
model_top3.add(Flatten(input_shape=train_features.shape[1:])) #Flatten layers are used when you got a multidimensional output and you want to make it linear to pass it onto a Dense layer.
model_top3.add(Dense(256, activation='relu')) #Dense layers are used when association can exist among any feature to any other feature in data point.
model_top3.add(Dropout(0.5)) #Dropout is a way of cutting too much association among features by dropping the weights (edges) at a probability.
model_top3.add(Dense(1, activation='sigmoid'))

# On compile :
model_top3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# On affiche :
model_top3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 25088)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               6422784   
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 6,423,041
Trainable params: 6,423,041
Non-trainable params: 0
_________________________________________________________________


In [51]:
# On definit les parametres pour l'entrainement :
epochs = 100

# On definit les callbacks : 
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy',mode='max',patience = 10,restore_best_weights=True,),
]

In [37]:
# On entraine le modèle:
historique3 = model_top3.fit(train_generator_bottleneck,
        epochs=epochs, 
        callbacks = my_callbacks,
        batch_size=batch_size,
        validation_data=validation_generator_bottleneck)

Epoch 1/100


ValueError: in user code:

    File "C:\Users\marti\anaconda3\envs\covid_cough\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\marti\anaconda3\envs\covid_cough\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\marti\anaconda3\envs\covid_cough\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\marti\anaconda3\envs\covid_cough\lib\site-packages\keras\engine\training.py", line 861, in train_step
        self._validate_target_and_loss(y, loss)
    File "C:\Users\marti\anaconda3\envs\covid_cough\lib\site-packages\keras\engine\training.py", line 818, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=binary_crossentropy, and th

In [ ]:
n_epochs = len(historique3.history['accuracy'])
GenOptimale3 = n_epochs-10
print("La generation optimale est : ",GenOptimale3)
print("Avec une accuracy de : ",historique3.history['accuracy'][GenOptimale3-1])
model_top3.save_weights('models/weights/vgg19_heavyTop_224_224_'+str(GenOptimale3)+'.h5')

In [ ]:
epochs = range(1,n_epochs+1)
plt.plot(epochs, historique3.history['accuracy'], label='Accuracy')
plt.plot(epochs, historique3.history['val_accuracy'], label='Validation Accuracy')
plt.plot(epochs, historique3.history['loss'], label='Loss')
plt.plot(epochs, historique3.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation loss and accuracy for the heavy layer fully connected')
plt.legend(loc="upper left")
plt.xlabel('Epochs')
plt.ylabel('Values')
plt.show()

In [ ]:
# On evalue :

# Avec les donnes de validation :
eval3 = model_top3.evaluate(validation_features, validation_labels,verbose=2)